In [2]:
import pandas as pd



# In[ ]:


chunks=pd.read_csv('eICU/vitalPeriodic.csv')


In [8]:
chunks[1000000:2000000].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 1000000 to 1999999
Data columns (total 19 columns):
vitalperiodicid      1000000 non-null int64
patientunitstayid    1000000 non-null int64
observationoffset    1000000 non-null int64
temperature          192449 non-null float64
sao2                 867348 non-null float64
heartrate            996666 non-null float64
respiration          732167 non-null float64
cvp                  139580 non-null float64
etco2                12323 non-null float64
systemicsystolic     302301 non-null float64
systemicdiastolic    302316 non-null float64
systemicmean         302839 non-null float64
pasystolic           46432 non-null float64
padiastolic          46419 non-null float64
pamean               46459 non-null float64
st1                  481038 non-null float64
st2                  486550 non-null float64
st3                  480329 non-null float64
icp                  3542 non-null float64
dtypes: float64(16), int64(3)
memor

In [5]:
chunks[-1000000:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 145671642 to 146671641
Data columns (total 19 columns):
vitalperiodicid      1000000 non-null int64
patientunitstayid    1000000 non-null int64
observationoffset    1000000 non-null int64
temperature          40165 non-null float64
sao2                 933473 non-null float64
heartrate            998144 non-null float64
respiration          762630 non-null float64
cvp                  93717 non-null float64
etco2                33550 non-null float64
systemicsystolic     235196 non-null float64
systemicdiastolic    235199 non-null float64
systemicmean         235197 non-null float64
pasystolic           26656 non-null float64
padiastolic          26657 non-null float64
pamean               26657 non-null float64
st1                  462623 non-null float64
st2                  472255 non-null float64
st3                  464073 non-null float64
icp                  11719 non-null float64
dtypes: float64(16), int64(3)
me

In [14]:
cf = chunks.dropna(subset=['icp'])[['vitalperiodicid', 'patientunitstayid']].patientunitstayid.unique()

In [5]:
cf = chunks.dropna(subset=['icp'])[['vitalperiodicid', 'patientunitstayid', 'observationoffset', 'systemicdiastolic', 'icp']]

In [6]:
col_names =  cf.columns
sorted_df  = pd.DataFrame(columns = col_names)
pids = cf.patientunitstayid.unique()
for pid in pids:
    sorted_df = sorted_df.append(cf[cf.patientunitstayid==pid].sort_values(by='observationoffset'))
sorted_df.reset_index(drop=True, inplace=True)

In [16]:
sorted_df[:1000000].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
vitalperiodicid      1000000 non-null object
patientunitstayid    1000000 non-null object
observationoffset    1000000 non-null object
systemicdiastolic    422950 non-null float64
icp                  1000000 non-null float64
dtypes: float64(2), object(3)
memory usage: 38.1+ MB


In [4]:
cf.head()

vitalperiodicid  patientunitstayid  systemicdiastolic   icp
54050         48697139             141800               50.0  23.0
54051         48676207             141800               46.0  32.0
54052         48688384             141800               46.0  29.0
54053         48690831             141800               52.0  26.0
54054         48700497             141800               42.0  18.0

In [ ]:
cf = chunks.dropna(subset=['icp'])[['vitalperiodicid', 'patientunitstayid', 'icp']]

In [11]:
pq = []
pids =  cf.patientunitstayid.unique()
for pid in pids:
    pq.append(float(len(cf[cf.patientunitstayid==pid])))


In [15]:
sum(pq)/len(pq)*5/60

134.19150346797224

In [23]:
pq = []
pids =  chunks.dropna(subset=['icp'])[['vitalperiodicid', 'patientunitstayid']].patientunitstayid.unique()
for pid in pids:
    pq.append(float(len(cf[cf.patientunitstayid==pid]))/len(chunks[chunks.patientunitstayid==pid]))
pq = sorted(pq)

In [1]:
q

NameError: name 'q' is not defined

In [21]:
q=[1,4,2,4]
sorted(q)

[1, 2, 4, 4]

In [28]:
pq[2*len(pq)/3]

0.8329774614472123

In [25]:
pq[4*len(pq)/5]

0.9272265707636979

In [ ]:
pids =  chunks.dropna(subset=['icp'])[['vitalperiodicid', 'patientunitstayid']].patientunitstayid.unique()
for pid in pids:
    print(len(cf[cf.patientunitstayid==pid]), len(chunks[chunks.patientunitstayid==pid]))

In [ ]:
len(chunks)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import pandas as pd
import numpy as np
import random as rnd
import os
import copy



# Preprocessing
from sklearn import preprocessing
import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score


# In[ ]:


time_priors = [2, 4, 6, 8]
time_duration = 36
time_sep = 24
noDataPoints = 90


# In[ ]:


chunks=pd.read_csv('eICU/vitalPeriodic.csv', chunksize=1000000)


# In[ ]:


def gety_fun(pos, time_prior, data_frame):
    tmp = data_frame.iloc[-1]['temperature']      
        
    data_frame = data_frame.iloc[:-pos-1][['sao2', 'heartrate', 'respiration', 'cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']]
    
    return data_frame.iloc[len(data_frame)-noDataPoints:-1].values, time_prior


# In[ ]:


dt = {}
med_df = pd.read_csv('eICU/medication.csv')
col_names =  med_df.columns
df  = pd.DataFrame(columns = col_names)
for pid in med_df.patientunitstayid.unique():
    df = df.append(med_df[med_df.patientunitstayid==pid].sort_values(by='drugstartoffset'))
df.reset_index(drop=True, inplace=True)
df = df.dropna(subset=['drugname'])
feverDrugs = ['paracetamol',  'acetaminophen', 'ibuprofen',  'tylenol', 'advil',  'aspirin',  'motrin',  'naproxen', 'aleve', 'bayer', 'aspirin',  'ibu']
for i in range(len(df)):
  #  print('iijoi', df.get(df.iloc[i]['patientunitstayid'], 0))
    if dt.get(df.iloc[i]['patientunitstayid'], [0,0])[0]==0:
        
        drugs = df.iloc[i]['drugname'].split()
        drugs = [drug.lower() for drug in drugs]
        if sum([int(feverDrug==drug) for feverDrug in feverDrugs for drug in drugs])>0:
            dt[df.iloc[i]['patientunitstayid']] = [1, df.iloc[i]['drugstartoffset']]
         #   print(i)
 


# In[ ]:


def process1(demo_df, drugdt):
    demo_df = demo_df[['vitalperiodicid', 'patientunitstayid', 'observationoffset', 'temperature', 'sao2', 'heartrate', 'respiration', 'cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']]
    demo_df[['vitalperiodicid','patientunitstayid', 'observationoffset']] = demo_df[['vitalperiodicid','patientunitstayid', 'observationoffset']].astype('int32')
    demo_df[['temperature','sao2', 'heartrate', 'respiration','cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']] = demo_df[['temperature','sao2', 'heartrate', 'respiration','cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']].astype('float32')
    
  #  demo_df['vitalperiodicid'] = demo_df['vitalperiodicid'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    
    
     
    temp = demo_df.dropna(subset=['temperature'])[['vitalperiodicid', 'patientunitstayid', 'observationoffset', 'temperature', 'sao2', 'heartrate', 'respiration', 'cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']]
    
    feverdf = copy.copy(temp)
    feverdf['temperature'] = feverdf['temperature'].apply(lambda x: np.nan if x<38.0 else x)
    feverdf = feverdf.dropna(subset=['temperature'])[['patientunitstayid']]
    
    feverpid = feverdf.patientunitstayid.unique()
    
    pospid = temp.patientunitstayid.unique()
    
    volume = len(temp.patientunitstayid.unique())
    #Removing those pid who does not have any fever data.
    demo_df = demo_df.drop(demo_df[[volume == Sum for Sum in sum([demo_df.patientunitstayid!=pid for pid in temp.patientunitstayid.unique()])]].index)
    
    dt = {}
    colms = ['systemicsystolic', 'systemicdiastolic', 'systemicmean', 'sao2', 'heartrate', 'respiration', 'cvp']
    for pid in demo_df.patientunitstayid.unique():
        for col in colms:
            dt[(pid, col)] = demo_df[demo_df['patientunitstayid']==pid][col].median()
    for pid in temp.patientunitstayid.unique():
        for col in colms:
            temp.loc[(temp.patientunitstayid==pid) & (temp[col].isnull()), col] = dt[(pid, col)]
    temp['systemicdiastolic'] = temp['systemicdiastolic'].fillna(demo_df['systemicdiastolic'].median())
    temp['systemicsystolic'] = temp['systemicsystolic'].fillna(demo_df['systemicsystolic'].median())
    temp['systemicmean'] = temp['systemicmean'].fillna(demo_df['systemicmean'].median())
    temp['sao2'] = temp['sao2'].fillna(demo_df['sao2'].median())
    temp['heartrate'] = temp['heartrate'].fillna(demo_df['heartrate'].median())
    temp['respiration'] = temp['respiration'].fillna(demo_df['respiration'].median())
    temp['cvp'] = temp['cvp'].fillna(demo_df['cvp'].median())
    
    for pid in demo_df.patientunitstayid.unique():
        for col in colms:
            demo_df.loc[(demo_df.patientunitstayid==pid) & (demo_df[col].isnull()), col] = dt[(pid, col)]

    demo_df['systemicdiastolic'] = demo_df['systemicdiastolic'].fillna(demo_df['systemicdiastolic'].median())
    demo_df['systemicsystolic'] = demo_df['systemicsystolic'].fillna(demo_df['systemicsystolic'].median())
    demo_df['systemicmean'] = demo_df['systemicmean'].fillna(demo_df['systemicmean'].median())
    demo_df['sao2'] = demo_df['sao2'].fillna(demo_df['sao2'].median())
    demo_df['heartrate'] = demo_df['heartrate'].fillna(demo_df['heartrate'].median())
    demo_df['respiration'] = demo_df['respiration'].fillna(demo_df['respiration'].median())
    demo_df['cvp'] = demo_df['cvp'].fillna(demo_df['cvp'].median())
    
    col_names =  demo_df.columns
    sorted_df  = pd.DataFrame(columns = col_names)
    for pid in demo_df.patientunitstayid.unique():
        sorted_df = sorted_df.append(demo_df[demo_df.patientunitstayid==pid].sort_values(by='observationoffset'))
    col_names =  temp.columns
    df  = pd.DataFrame(columns = col_names)
    for pid in temp.patientunitstayid.unique():
        df = df.append(temp[temp.patientunitstayid==pid].sort_values(by='observationoffset'))
    sorted_df.reset_index(drop=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    df_epCount  = pd.DataFrame(columns = ['epCount'])
    for pid in sorted_df.patientunitstayid.unique():
       # print(pid)
        df_fever = sorted_df[sorted_df.patientunitstayid==pid]['temperature'].apply(lambda x: 1 if x>=38.0 else 0)        
        df_epCount = pd.concat([df_epCount, (df_fever.cumsum()-df_fever).apply(lambda x: str(x))])
    
    sorted_df['patientunitstayid'] = sorted_df['patientunitstayid'].apply( lambda x: str(x))
    sorted_df['epCount'] = df_epCount[0]
    sorted_df['patientunitstayid'] =  sorted_df[['patientunitstayid', 'epCount']].apply(lambda x: '_'.join(x), axis=1)
    colm = ['sao2', 'heartrate', 'respiration', 'cvp', 'systemicsystolic', 'systemicdiastolic', 'systemicmean']
    
    
    x=[]
    y=[]
    z=[]
    for clmn in colm:
        mean = sorted_df[clmn].mean()
        std = sorted_df[clmn].std()
        sorted_df[clmn] = sorted_df[clmn].apply(lambda x: (x-mean)/std)
    
   
        
    temperature_list = sorted_df[sorted_df['temperature'].notnull()].index.tolist()
    i=0
    prev_idx = 0
    pid = sorted_df.iloc[0]['patientunitstayid']
    lastFever = sorted_df.iloc[0]['observationoffset']# does not matter
    prev_pid = -1
    freq = 1/5

    for idx in temperature_list: 
        pid = sorted_df.iloc[idx]['patientunitstayid']
        if not pid==prev_pid:
            if not prev_pid==-1:
                lastFever = sorted_df[sorted_df['patientunitstayid'] == prev_pid].iloc[-1]['observationoffset']
            prev_pid = sorted_df.iloc[idx]['patientunitstayid']
            prev_idx = sorted_df.index[sorted_df['patientunitstayid'] == pid].tolist()[0]
            
        if pid not in feverpid and sorted_df.iloc[idx]['temperature']<38.0:
            pos = 0
            if len(sorted_df[prev_idx:idx+1].iloc[:-pos-1])<=noDataPoints or drugdt.get(pid, [0, 0])[0]==1 or np.random.rand()>0.01:
                pass
            else:
                oop = gety_fun(pos, -1, sorted_df[prev_idx:idx+1])
                x.append(oop[0])
                y.append(oop[1])
                z.append(pid)
        
        
        elif (not sorted_df.loc[idx]['epCount'] == '0') or drugdt.get(pid, [0, 0])[1]<sorted_df.iloc[idx]['observationoffset']:        
        
            if sorted_df.loc[idx]['epCount'] == '0' or sorted_df.loc[idx]['observationoffset'] - lastFever > time_sep*60:

        
                while sorted_df.iloc[idx]['observationoffset'] - sorted_df.iloc[prev_idx]['observationoffset'] > time_duration*60:
                    prev_idx+=1
 #   print(prev_idx, idx)
                time_diff = sorted_df.loc[idx]['observationoffset'] - sorted_df.loc[prev_idx]['observationoffset']
    
  #  print(time_diff)

                for time_prior in time_priors:
                    if time_diff < time_prior*60:
                        pass
    
                    else:
               # freq = len(sorted_df[prev_idx:idx+1])/time_diff.total_seconds()
                        pos = int(60*time_prior*freq)
                        if len(sorted_df[prev_idx:idx+1].iloc[:-pos-1])<=noDataPoints:
                            pass
                        else:
                            oop = gety_fun(pos, time_prior, sorted_df[prev_idx:idx+1])
                            x.append(oop[0])
                            y.append(oop[1])
                            z.append(pid)
        i+=1
    
    y = np.array(y)
    x=np.array(x)
    return y, x, z, pospid


# In[ ]:


iu = 0
Y , X, Z = [], [], []
pospid = []
for chunk in chunks:
    iu+=1
    y, x, z, pid= process1(chunk, dt)
    pospid.append(pid)
    print(iu)
   # print(len(pid), sum(y), len(y))
    for i in range(len(y)):
        Y.append(y[i])
        X.append(x[i])
        Z.append(z[i])


# In[ ]:


np.save('TCNRegressionobsfullDX', X)
np.save('TCNRegressionobsfullDY', Y)
np.save('TCNRegressionobsfullDZ', Z)
np.save('TCNRegressionobsfullDpospid', pospid)



/sw/acf/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [1]:
import keras

Using TensorFlow backend.


In [2]:
#import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense